- Inspired from demo.ipynb in NGSIM.jl
- Takeaways
    - 101 has 5 segments
    - Start from bottom of screen, 1 is bottom of screen, 2 is exit ramp,
    3 is straight section, 4 is entry to road till entry ramp 
    join point, 5 is entry ramp

- For `observe!` from `Tim2DDriver` to work, had to make the following changes
    - Had to modify `AutomotiveDrivingModels.jl/src/2d/vehicles/scene_records.jl` to extend
the default length of the capacity in the record container from 100 to 300
    - Did the same capacity extension in `Records.jl/src/frames.jl`, `QueueRecords.jl` and 
`ListRecords.jl`
- Note that querying Tim2DDriver yields a `latLonAccel` return type. Had to create an array
to enable these values to work with `step`

In [1]:
using NGSIM
using AutomotiveDrivingModels
using AutoViz
using Interact # For slider bar trajectory propagation
using Reel

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/5dc1e185f8b2175037a0bb4bac5baa1c8ca39ea5-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/5dc1e185f8b2175037a0bb4bac5baa1c8ca39ea5-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
roadway = ROADWAY_101;

In [3]:
using AutoEnvs

n_veh = 1
filepath = Pkg.dir("NGSIM", "data", 
    "trajdata_i101_trajectories-0805am-0820am.txt")
params = Dict(
        "trajectory_filepaths"=>[filepath],
        "H"=>200,
        "primesteps"=>50,
        "n_veh"=>n_veh,
        "remove_ngsim_veh"=>false
)
# env = MultiagentNGSIMEnvVideoMaker(params)
env = MultiagentNGSIMEnv(params);
timestep = 0.1;

INFO: Recompiling stale cache file /home/raunak/.julia/lib/v0.6/AutoEnvs.ji for module AutoEnvs.


In [ ]:
model = Tim2DDriver(timestep,mlane = MOBIL(timestep));

In [5]:
mlon = IntelligentDriverModel(σ=0.1,k_spd=1.0,T=0.1,s_min=2.0,
    a_max=3.0,d_cmf=2.5)
mlat = ProportionalLaneTracker(σ=0.1, kp=3.0, kd=2.0)
mlane = MOBIL(timestep,politeness=0.1,advantage_threshold=0.01)

model = Tim2DDriver(timestep, mlon=mlon, mlat=mlat, mlane=mlane);

In [8]:
# This will create a series of images in /tmp/episode1
# You can open the first and then press right arrow

# a = ones(n_veh, 2)
# a[2] = 0.0
reset(env,random_seed=3);
t_init = env.t
for _ in 1:200
    render(env)
    observe!(model, env.scene, env.roadway, env.ego_vehs[1].id)
    latlonacc = rand(model)
#     a = [latlonacc.a_lon latlonacc.a_lat] # This form for AccelTurnRate
    a = [latlonacc.a_lat latlonacc.a_lon]   # This form for LatLonAccel
    @show env.t-t_init a
    
    # Don't we need accelturnrate for the env?
    x, r, terminal, _ = step(env, a)
end

env.t - t_init = 0
a = [2.13805 2.94439]
env.t - t_init = 1
a = [1.7772 3.02039]
env.t - t_init = 2
a = [1.35391 2.87776]
env.t - t_init = 3
a = [0.813099 2.92052]
env.t - t_init = 4
a = [0.461407 2.74103]
env.t - t_init = 5
a = [0.290203 2.81288]
env.t - t_init = 6
a = [-0.128257 2.8215]
env.t - t_init = 7
a = [-0.251965 2.82144]
env.t - t_init = 8
a = [-0.394649 2.85068]
env.t - t_init = 9
a = [-0.452823 2.91957]
env.t - t_init = 10
a = [-0.462927 2.6814]
env.t - t_init = 11
a = [-0.496535 2.75792]
env.t - t_init = 12
a = [-0.670195 2.7036]
env.t - t_init = 13
a = [-0.515846 2.56239]
env.t - t_init = 14
a = [-0.535674 2.51967]
env.t - t_init = 15
a = [-0.441624 2.30045]
env.t - t_init = 16
a = [-0.581775 2.40706]
env.t - t_init = 17
a = [-0.644537 2.35591]
env.t - t_init = 18
a = [-0.37395 2.04942]
env.t - t_init = 19
a = [-0.197892 1.88066]
env.t - t_init = 20
a = [-0.399568 1.81542]
env.t - t_init = 21
a = [-0.146147 1.74485]
env.t - t_init = 22
a = [-0.291761 1.54478]
env.t - t_in

env.t - t_init = 181
a = [-0.0201828 0.155138]
env.t - t_init = 182
a = [0.110704 0.0993388]
env.t - t_init = 183
a = [0.00910817 0.281509]
env.t - t_init = 184
a = [-0.200263 0.148946]
env.t - t_init = 185
a = [0.106294 0.336552]
env.t - t_init = 186
a = [-0.186111 0.251934]
env.t - t_init = 187
a = [-0.0209165 0.293927]
env.t - t_init = 188
a = [0.077382 0.233975]
env.t - t_init = 189
a = [-0.0673854 0.498495]
env.t - t_init = 190
a = [-0.181191 0.556625]
env.t - t_init = 191
a = [0.225981 0.674542]
env.t - t_init = 192
a = [-0.0305838 0.746174]
env.t - t_init = 193
a = [0.00544823 0.591234]
env.t - t_init = 194
a = [-0.0935109 1.05159]
env.t - t_init = 195
a = [0.147424 0.791708]
env.t - t_init = 196
a = [0.011235 0.841019]
env.t - t_init = 197
a = [-0.114132 1.02136]
env.t - t_init = 198
a = [-0.0439777 1.30344]
env.t - t_init = 199
a = [-0.212779 1.1543]


In [ ]:
function makevid(t,dt)
    for _ in t:300
    render(env)
    observe!(model, env.scene, env.roadway, env.ego_vehs[1].id)
    latlonacc = rand(model)
    a = [latlonacc.a_lon latlonacc.a_lat]
    x, r, terminal, _ = step(env, a)
    end
end

In [ ]:
reset(env,random_seed=4)
film = roll(makevid, fps=10, duration=1.0)

In [ ]:
write("mygif.gif",film)

In [ ]:
# Take a look at sisl/gail-driver/julia/envs/Auto2D.jl function reel_drive
# for video making inspiration

## Video making using recorded cars on ngsim
- Trajdatas is what the car trajectories are stored in
- We want to color the ego vehicle differently to be able to see it

In [ ]:
# argument 1 loads i101 7:50 to 8:05.
# load_trajdata function defined in NGSIM.jl/src/trajdata.jl
td1 = load_trajdata(1); 

scene = Scene(500)
egoid = 546

# Drive here in the notebook. Replay the trajectory as recorded in the ngsim data
@manipulate for i in 1000:2000
    temp_scene = get!(scene,td1,i)
    
    carcolors = Dict{Int,Colorant}()
    for veh in temp_scene
        #@show veh.id
        # if veh id matches the egoid color it blue otherwise green
        carcolors[veh.id] = 
        in(veh.id, egoid) ? colorant"blue" : colorant"green"
    end
    render(temp_scene, ROADWAY_101, 
        cam=CarFollowCamera{Int}(546,5.0),
#         cam=StaticCamera(VecE2(1966400, 570900), 5.0),
#         cam=FitToContentCamera(0.),
        car_colors=carcolors)
end